# A simple agent based infection model with Mesa and Bokeh

This shall serve the purpose of educating and leveraging mesa + agm with a project based approach<br>
refs: <a>https://dmnfarrell.github.io/bioinformatics/abm-mesa-python</a>

In [7]:
import time
import numpy as np
from enum import IntEnum
import pandas as pd
import collections
import pandas as pd
import pylab as plt  # Watch out here
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector


### Building a simple model
<br>
The main idea with Mesa is to create two classes, one for the model and the other for the agents. The agent handles the behavior of the individual being simulated such as how it can infect neighbors in a grid or network. The model holds all the general parameters, a grid object for moving agents on and it also creates and tracks it's agents. It's really much more instructive to go through an example than describe. This code was made mostly using the Mesa tutorial on Virus on network example.<br><br>
We first make a Model class defining a grid, scheduler for tracking the order of agents being activated in time. Time periods are represented as steps and the agents can all move once in each step. The the agents will decide if it can infect another according to where it is. The DataCollector class keeps track of agent information through the simulation. The grid is a MultiGrid class, which let more than one agent occupy a cell at once.

In [9]:
class InfectionModel(Model):
    '''A model for infection spread.'''
    def __init__(self, N = 10, width = 10, height = 10, ptrans = 0.5, death_rate = 0.02, recovery_days = 21, recovery_sd = 7):
        self.num_agents = N
        self.recovery_days = recovery_days
        self.recovery_sd = recovery_sd
        self.ptrans = ptrans
        self.death_rate = death_rate
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.running = True
        self.dead_agents = []
        # Create agents
        for i in range(self.num_agents):
            a = MyAgent(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.rand.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))
            # Make some agents infected at start
            infected = np.random.choice([0, 1], p = [0.98, 0.02])
            if infected == 1:
                a.state = State.INFECTED
                a.recovery_time = self.get_recovery_time()

        self.datacollector = DataCollector(
            agent_reporters={"State": "state"})

    def get_recovery_time(self):
        return int(self.random.normalvariate(self.recovery, self.recovery_sd))
    
    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()
    

We then create the Agent class. It has three possible states and transitions between them through the simulation. At each step the agent will move and then can carry out any operation such as infecting another agent in the same cell in the grid if the other agent is susceptible. The agent can also recover over time.

In [8]:
class State(IntEnum):
    SUSCEPTIBLE = 0 
    INFECTED = 1
    REMOVED = 2
    
    
class MyAgent(Agent):
    '''An agent in an epidemic model.'''
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.age = self.random.normalvariate(20, 40)
        self.state = State.SUSCEPTIBLE
        self.infection_time = 0
        
        
    def move(self):
        '''Move the agent.'''
        
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center = False)
        
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)
    
    
    def status(self):
        '''Check infection status.'''
        
        if self.state == State.INFECTED:
            drate = self.model.death_rate
            alive = np.random.choice([0,1], p=[drate, 1-drate])
            if alive == 0:
                self.model.schedule.remove(self)
            t = self.model.schedule.time - self.infection_time
            if t >= self.recovery_time:
                self.state = State.REMOVED
                
    
    def contact(self):
        '''Find close contacts and infect.'''
        
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        if len(cellmates) > 1:
            for other in cellmates:
                if self.random.random() > model.ptrans:
                    continue
                if self.state is State.INFECTED and other.state is State.SUSCEPTIBLE:
                    other.state = State.INFECTED
                    other.infection_time = self.model.schedule.time
                    other.recovery_time = model.get_recovery_time()
                    
    def stop(self):
        self.status()
        self.move()
        self.contact()
        

### Run the model
We can now run the model by simply iterating over the number of steps we want. The DataCollector object has stored agent variables along the way and this can be analysed to get model results. get_agent_vars_dataframe() returns a pandas DataFrame in long form of the state of each agent at each step.